In [1]:
import numpy as np
import h5py

In [2]:
def get_cpsi(num_phi, wavlen, q1, q2):
        """
        Returns cos psi values
        ================================================================
        num_phi  - int, number of phi values in the ring
        
        wavlen   - float, wavelength in angstrom
        
        q1, q2   - float, magnitudes of q values
        
        Returns:
        cpsi     - 1d array, shape = (num_phi)
        """
        phis = np.arange( num_phi ) * 2 * np.pi / num_phi
        th1 = np.arcsin( q1 * wavlen / 4. / np.pi )
        th2 = np.arcsin( q2 * wavlen / 4. / np.pi )
        cpsi = np.cos(phis) * np.cos(th1)*np.cos(th2) + np.sin(th1)*np.sin(th2)
        
        return cpsi

In [3]:
f = h5py.File('/reg/d/psdm/cxi/cxilr6716/scratch/combined_tables/gai/run108.tbl','r')
print f.keys()
f['ebeam']['photon_energy'].value.mean(0)

[u'ebeam', u'event_time', u'evr', u'fiducials', u'gas_detector', u'phase_cav', u'polar_imgs', u'polar_mask', u'polar_mask_binned', u'polar_params', u'radial_profs']


9496.1567184952964

In [13]:
phi_offset = 10
waxs_rmin = 65 # pixel units
waxs_rmax = 415
c = 2.998e8
h = 6.63e-34
ev_to_j = 1.602e-19
num_q = 50

pixsize = 110e-6 #meter
photon_energy = 9.496e3 #eV
wavlen = c * h/(photon_energy * ev_to_j) * 1e10 # angstrom
det_dist = 121e-3 # meter

thetas = np.arctan ( np.arange(waxs_rmin, waxs_rmax) * pixsize / det_dist)  /2.    
qs =np.sin( thetas ) * 4.*np.pi/wavlen
print qs.min(), qs.max()
qs.shape
q_intervals = np.zeros( (num_q,2) )
interval = 7
# interval = int ( (waxs_rmax-waxs_rmin)/(num_q) )
for idx in range(num_q):
    try:
        q_intervals[idx] = np.array([qs[idx*interval],qs[(idx+1)*interval] ])
    except IndexError:
        q_intervals[idx] = np.array([qs[idx*interval],qs[-1] ])
        
qvalue = q_intervals[:,0]
print qvalue.shape

print qvalue[:36]


0.283785919305 1.72167712864
(50,)
[ 0.28378592  0.3142544   0.34469446  0.3751034   0.40547854  0.4358172
  0.46611672  0.49637448  0.52658787  0.55675429  0.58687118  0.61693601
  0.64694625  0.67689942  0.70679306  0.73662475  0.76639207  0.79609267
  0.8257242   0.85528437  0.88477091  0.91418157  0.94351417  0.97276654
  1.00193656  1.03102213  1.06002121  1.0889318   1.11775191  1.14647962
  1.17511304  1.20365032  1.23208966  1.26042928  1.28866747  1.31680255]


In [8]:
num_phi = 326
cpsi = np.zeros( (qvalue.size, num_phi) )
for idx in range(qvalue.size):
    cpsi[idx] = get_cpsi(num_phi,wavlen,qvalue[idx],qvalue[idx])

In [15]:
# np.save('encodeZ400_cpsi.npy',cpsi)
# np.save('encodeZ400_qvalue.npy',qvalue)

In [16]:
cpsi2 = np.load('encodeZ400_cpsi.npy')
qvalue2 = np.load('encodeZ400_qvalue.npy')

In [17]:
qvalue2

array([ 0.28378592,  0.3142544 ,  0.34469446,  0.3751034 ,  0.40547854,
        0.4358172 ,  0.46611672,  0.49637448,  0.52658787,  0.55675429,
        0.58687118,  0.61693601,  0.64694625,  0.67689942,  0.70679306,
        0.73662475,  0.76639207,  0.79609267,  0.8257242 ,  0.85528437,
        0.88477091,  0.91418157,  0.94351417,  0.97276654,  1.00193656,
        1.03102213,  1.06002121,  1.0889318 ,  1.11775191,  1.14647962,
        1.17511304,  1.20365032,  1.23208966,  1.26042928,  1.28866747,
        1.31680255,  1.34483287,  1.37275684,  1.4005729 ,  1.42827956,
        1.45587534,  1.48335882,  1.51072862,  1.5379834 ,  1.56512187,
        1.59214279,  1.61904494,  1.64582716,  1.67248833,  1.69902737])

In [14]:
print wavlen

1.30659784999
